In [1]:
! pip install PyPDF2
! pip install chromadb
! pip install tavily
! pip install nemoguardrails
! pip install -U langchain-openai

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_core-1.2.1-py3-none-any.whl.metadata (3.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 3.5 MB/s eta 0:00:00
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
Using cached langchain_core-1.2.1-py3-none-any.whl (475 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.1/413.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsmith-0.0.87
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.20
    Uninstalling langchain-community-0.0.20:
      Successfully uninstalled langchain-community-0.0.20
  Atte

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 2.4 MB/s eta 0:00:00


In [2]:
#pip install langchain==0.0.353

In [2]:
from openai import OpenAI
import gradio as gr
import os
import PyPDF2
import chromadb
from dotenv import load_dotenv
load_dotenv()
import requests
from urllib.parse import urlparse
import re
import json
from datetime import datetime
import uuid
from sentence_transformers import SentenceTransformer, util
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
os.environ["OPENAI_API_KEY"] = "" #Add your OPENAI API Key
load_dotenv()
from openai import OpenAI
client = OpenAI(api_key="")

In [4]:
# LangSmith imports with error handling
os.environ["LANGCHAIN_API_KEY"] = ""  #Add your LANGCHAIN API Key
try:
    import langsmith
    from langsmith import traceable
    LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
    if LANGCHAIN_API_KEY:
        os.environ["LANGCHAIN_TRACING_V2"] = "true"
        os.environ["LANGCHAIN_PROJECT"] = "physics-teacher-bot"
        langsmith_client = langsmith.Client()
        LANGCHAIN_AVAILABLE = True
        print("✅ LangSmith tracing initialized successfully!")
    else:
        print("⚠️  LANGCHAIN_API_KEY not found. LangSmith tracing will be disabled.")
        langsmith_client = None
        LANGCHAIN_AVAILABLE = False
except ImportError:
    print("⚠️  LangSmith not installed. Tracing will be disabled.")
    print("💡 Run: pip install langsmith")
    langsmith_client = None
    LANGCHAIN_AVAILABLE = False
except Exception as e:
    print(f"⚠️  LangSmith initialization failed: {e}")
    langsmith_client = None
    LANGCHAIN_AVAILABLE = False

✅ LangSmith tracing initialized successfully!


In [5]:
# Initialize Tavily client with error handling
os.environ["TAVILY_API_KEY"] = ""  #Add your TAVILY API Key
try:
    from tavily import TavilyClient
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    TAVILY_AVAILABLE = True
except ImportError:
    TAVILY_AVAILABLE = False
    print("⚠️  Tavily not installed. Web search functionality will be disabled.")
    print("💡 Run: pip install tavily-python")

In [6]:
# Initialize NVIDIA NeMo Guardrails with error handling
try:
    from nemoguardrails import RailsConfig, LLMRails
    from nemoguardrails.actions import action
    NEMO_GUARDRAILS_AVAILABLE = True
except ImportError:
    NEMO_GUARDRAILS_AVAILABLE = False
    print("⚠️  NVIDIA NeMo Guardrails not installed. Safety features will be disabled.")
    print("💡 Run: pip install nemoguardrails")

In [7]:
# Initialize ChromaDB
chroma_client = chromadb.Client()
pdf_collection = chroma_client.create_collection(name="PhysicsPDFs")
memory_collection = chroma_client.create_collection(name="ConversationMemory")

In [8]:
import nest_asyncio
nest_asyncio.apply()
print("✅ Enabled nested asyncio; sync guardrails.generate() is now allowed.")

✅ Enabled nested asyncio; sync guardrails.generate() is now allowed.


In [9]:
# Initialize NeMo Guardrails
from nemoguardrails import LLMRails, RailsConfig

assert "OPENAI_API_KEY" in os.environ, "Set OPENAI_API_KEY in environment."

# Load config from file
config = RailsConfig.from_path("rails")
#print("Models in config:", config.as_dict().get("models"))
nemo_rails = LLMRails(config)
print("✅ NeMo Guardrails initialized from rails/")

def apply_guardrails(user_message: str, bot_response: str = ""):
    """
    Apply NeMo Guardrails for safety and content filtering.
    Returns (allowed: bool, message: str).
    - If blocked, message is the guardrails response.
    - If allowed, message is either the original bot_response or 'OK'.
    """
    if not NEMO_GUARDRAILS_AVAILABLE:
        return apply_keyword_safety_check(user_message, bot_response)

    try:
        # Use NeMo Guardrails for safety checking
        guarded_reply = nemo_rails.generate(
            messages=[{"role": "user", "content": user_message}]
        )
        # Extract text safely
        guarded_text = _extract_text_from_reply(guarded_reply)

        # If guardrails produced a block message, return blocked
        if "I can’t help with harmful, dangerous, or illegal topics." in guarded_text:
            return False, guarded_text
        if "I’m a physics tutor. Please ask science or education topics." in guarded_text:
            return False, guarded_text

        # Otherwise, if bot_response was passed, return it; else 'OK'
        return True, bot_response or "OK"

    except Exception as e:
        print(f"Guardrails error: {e}")
        # Fallback to keyword-based safety check
        return apply_keyword_safety_check(user_message, bot_response)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

model.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

✅ NeMo Guardrails initialized from rails/


In [10]:
class TokenUsageTracker:
    """Track token usage across different components"""

    def __init__(self):
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.costs = 0.0

    def update_from_response(self, response, model="gpt-3.5-turbo"):
        """Update token counts from OpenAI response"""
        if hasattr(response, 'usage'):
            usage = response.usage
            self.prompt_tokens += getattr(usage, 'prompt_tokens', 0)
            self.completion_tokens += getattr(usage, 'completion_tokens', 0)
            self.total_tokens += getattr(usage, 'total_tokens', 0)

            # Calculate cost (approximate)
            cost_rates = {
                "gpt-3.5-turbo": (0.0015, 0.002),  # $0.0015/1K input, $0.002/1K output
                "gpt-4": (0.03, 0.06),  # $0.03/1K input, $0.06/1K output
                "gpt-4-turbo": (0.01, 0.03)  # $0.01/1K input, $0.03/1K output
            }

            input_rate, output_rate = cost_rates.get(model, (0.0015, 0.002))
            self.costs += (self.prompt_tokens / 1000 * input_rate) + (self.completion_tokens / 1000 * output_rate)

    def get_summary(self):
        """Get token usage summary"""
        return {
            "total_tokens": self.total_tokens,
            "prompt_tokens": self.prompt_tokens,
            "completion_tokens": self.completion_tokens,
            "estimated_cost": round(self.costs, 4)
        }

In [11]:
# Global token tracker
token_tracker = TokenUsageTracker()

def traceable_function(func):
    """Decorator that conditionally applies tracing based on availability"""
    if LANGCHAIN_AVAILABLE:
        return traceable(func)
    return func

In [12]:
@traceable_function
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [13]:
@traceable_function
def chunk_text(text, chunk_size=1000):
    """Split text into chunks"""
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [14]:
@traceable_function
def store_pdf_in_db(pdf_path):
    """Extract, chunk and store PDF in ChromaDB"""
    try:
        text = extract_text_from_pdf(pdf_path)
        chunks = chunk_text(text)

        # Clear existing data
        pdf_collection.delete(where={"source": {"$ne": ""}})

        # Add chunks to ChromaDB
        for i, chunk in enumerate(chunks):
            pdf_collection.add(
                documents=[chunk],
                metadatas=[{"source": pdf_path, "chunk_id": i, "type": "pdf"}],
                ids=[f"pdf_chunk_{i}_{os.path.basename(pdf_path)}"]
            )

        # Log to LangSmith
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=None,
                    key="pdf_processing",
                    score=1.0,
                    comment=f"Processed {len(chunks)} chunks from {pdf_path}"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return f"✅ Successfully loaded {len(chunks)} chunks from {pdf_path}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

In [15]:
@traceable_function
def get_relevant_context(query, n_results=3):
    """Retrieve relevant context from PDF knowledge base"""
    try:
        results = pdf_collection.query(
            query_texts=[query],
            n_results=n_results
        )
        return "\n\n".join(results['documents'][0]) if results['documents'] else ""
    except Exception as e:
        print(f"PDF context query error: {e}")
        return ""

In [16]:
@traceable_function
def store_conversation_memory(user_message, bot_response, conversation_context=""):
    """Store conversation in long-term memory"""
    try:
        # Create a memory entry
        memory_id = str(uuid.uuid4())
        timestamp = datetime.now().isoformat()

        # Combine user message and bot response for semantic search
        memory_text = f"User: {user_message}\nBot: {bot_response}"

        if conversation_context:
            memory_text = f"Context: {conversation_context}\n{memory_text}"

        # Store in memory collection
        memory_collection.add(
            documents=[memory_text],
            metadatas=[{
                "type": "conversation",
                "timestamp": timestamp,
                "user_message": user_message,
                "bot_response": bot_response,
                "conversation_context": conversation_context
            }],
            ids=[memory_id]
        )

        return True
    except Exception as e:
        print(f"Memory storage error: {e}")
        return False

In [17]:
@traceable_function
def get_relevant_memories(query, n_results=2):
    """Retrieve relevant past conversations from memory"""
    try:
        results = memory_collection.query(
            query_texts=[query],
            n_results=n_results
        )

        memories = []
        if results['documents']:
            for i, doc in enumerate(results['documents'][0]):
                metadata = results['metadatas'][0][i]
                memory_text = f"Previous conversation ({metadata['timestamp'][:10]}): {doc}"
                memories.append(memory_text)

        return "\n\n".join(memories) if memories else ""
    except Exception as e:
        print(f"Memory retrieval error: {e}")
        return ""

In [18]:
@traceable_function
def get_conversation_summary(history):
    """Generate a summary of the current conversation context"""
    if not history:
        return ""

    # Take last few exchanges for context
    recent_history = history[-3:] if len(history) > 3 else history
    context_parts = []

    for exchange in recent_history:
        if isinstance(exchange, tuple) and len(exchange) == 2:
            user_msg, bot_msg = exchange
            context_parts.append(f"User: {user_msg}")
            context_parts.append(f"Bot: {bot_msg}")
        elif isinstance(exchange, list) and len(exchange) == 2:
            user_msg, bot_msg = exchange
            context_parts.append(f"User: {user_msg}")
            context_parts.append(f"Bot: {bot_msg}")

    return "\n".join(context_parts)

In [19]:
@traceable_function
def is_pdf_context_relevant(pdf_context, query):
    """Check if PDF context is actually relevant to the query"""
    if not pdf_context or len(pdf_context.strip()) < 50:
        return False

    # Check for common irrelevant patterns in PDF context
    irrelevant_patterns = [
        "table of contents", "copyright", "abstract", "references",
        "appendix", "index", "acknowledg", "preface"
    ]

    pdf_lower = pdf_context.lower()
    query_lower = query.lower()

    # If PDF context contains many irrelevant sections, it's likely not useful
    irrelevant_count = sum(1 for pattern in irrelevant_patterns if pattern in pdf_lower)
    if irrelevant_count > 2:
        return False

    # Check if query terms actually appear in the context
    query_terms = query_lower.split()
    relevant_terms = sum(1 for term in query_terms if len(term) > 3 and term in pdf_lower)

    # If less than 30% of meaningful query terms are in context, consider it irrelevant
    meaningful_terms = [term for term in query_terms if len(term) > 3]
    if meaningful_terms and (relevant_terms / len(meaningful_terms)) < 0.3:
        return False

    return True

In [21]:
@traceable_function
def apply_keyword_safety_check(user_message, bot_response):
    """Fallback keyword-based safety check"""
    harmful_keywords = [
        "bomb", "weapon", "explosive", "harmful", "dangerous", "illegal",
        "hack", "cheat", "bypass", "malicious", "violence", "hurt", "kill"
    ]

    user_lower = user_message.lower()
    response_lower = bot_response.lower()

    # Check for harmful content in user message
    if any(keyword in user_lower for keyword in harmful_keywords):
        safety_message = "I apologize, but I cannot engage with that type of question. I'm here to help with physics education and related scientific topics."
        return safety_message, "🛡️ Input blocked by safety check"

    # Check for harmful content in bot response
    if any(keyword in response_lower for keyword in harmful_keywords):
        safety_message = "I need to provide a safe and educational response. Let me rephrase that to focus on the educational aspects of physics."
        return safety_message, "🛡️ Response moderated by safety check"

    return bot_response, "🛡️ Safety check passed"

In [54]:
def _extract_text_from_reply(reply):
    if reply is None:
        return ""
    if isinstance(reply, str):
        return reply
    if isinstance(reply, dict):
        for k in ("content", "message", "text"):
            if isinstance(reply.get(k), str):
                return reply[k]
        choices = reply.get("choices") or reply.get("outputs")
        if isinstance(choices, list) and choices:
            first = choices[0]
            if isinstance(first, dict):
                return first.get("message", {}).get("content") or first.get("content", "")
    if isinstance(reply, (list, tuple)) and reply:
        return _extract_text_from_reply(reply[0])
    return str(reply)

In [23]:
import csv
from pathlib import Path
from datetime import datetime

LOGS_DIR = Path("logs")
LOGS_DIR.mkdir(exist_ok=True)
DIAG_CSV = LOGS_DIR / "diagnostics.csv"

if not DIAG_CSV.exists():
    with DIAG_CSV.open("w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            "timestamp", "user_message", "allowed", "clarify", "generator",
            "edu_topk_mean", "edu_max", "harm_topk_mean", "harm_max",
            "len_tokens", "answer_snippet", "raw_reply_summary"
        ])

def _safe_get(d: dict, key: str, default=None):
    try:
        return d.get(key, default) if isinstance(d, dict) else default
    except Exception:
        return default

def log_diagnostics_csv(user_message: str, allowed: bool, clarify: bool, generator: str,
                        diagnostics: dict, answer_text: str, raw_reply: object):
    """
    Append a single diagnostics row. Non-blocking: exceptions are swallowed.
    Keep answer_snippet short to avoid huge CSV cells.
    """
    try:
        ts = datetime.utcnow().isoformat() + "Z"
        edu_topk = _safe_get(diagnostics, "edu_topk_mean", "")
        edu_max = _safe_get(diagnostics, "edu_max", "")
        harm_topk = _safe_get(diagnostics, "harm_topk_mean", "")
        harm_max = _safe_get(diagnostics, "harm_max", "")
        length = _safe_get(diagnostics, "len_tokens", "")
        snippet = (answer_text or "")[:400].replace("\n", " ").strip()
        raw_summary = ""
        try:
            if isinstance(raw_reply, dict):
                raw_summary = raw_reply.get("content") or str({k: raw_reply.get(k) for k in ("role","type") if k in raw_reply})
            elif isinstance(raw_reply, (list, tuple)):
                first = raw_reply[0]
                if isinstance(first, dict):
                    raw_summary = first.get("content", "")[:200]
                else:
                    raw_summary = str(first)[:200]
            else:
                raw_summary = str(raw_reply)[:200]
        except Exception:
            raw_summary = ""
        row = [
            ts, user_message, str(bool(allowed)), str(bool(clarify)), generator,
            edu_topk, edu_max, harm_topk, harm_max, length, snippet, raw_summary
        ]
        with DIAG_CSV.open("a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(row)
    except Exception:
        pass

print("✅ diagnostics CSV logger ready:", DIAG_CSV)


✅ diagnostics CSV logger ready: logs/diagnostics.csv


In [24]:
def rewrite_ambiguous_query_with_llm(user_message: str, history: list, max_history_turns: int = 4):
    """
    Ask the LLM to rewrite a short/ambiguous user_message into an explicit query
    using the last few turns. Returns the rewritten string or the original message.
    """
    recent = []
    for turn in history[-max_history_turns:]:
        if isinstance(turn, (list, tuple)) and len(turn) >= 2:
            recent.append(f"User: {turn[0]}\nAssistant: {turn[1]}")
    context_block = "\n\n".join(recent)

    rewrite_prompt = (
        "You are a concise assistant that rewrites short or pronoun-based follow-up "
        "questions into explicit, self-contained questions using the conversation history. "
        "Do not add new facts. If the user message is already explicit, return it unchanged.\n\n"
        f"Conversation history:\n{context_block}\n\n"
        f"User follow-up: {user_message}\n\n"
        "Return a single-line rewritten question that is explicit (for example: "
        "'dimensions of resistance and resistivity')."
    )

    rewritten, raw, src = generate_answer(rewrite_prompt, history=[], clarify=False, prefer_generator="nemo")
    if not rewritten or len(rewritten.strip()) < 3 or "{user_message}" in rewritten:
        return user_message
    return rewritten.strip()

In [50]:
def build_clarification_payload(user_message, memory_summary="", diagnostics=None):
    body = prompt_mgr.render_clarify_user(
        original_query=user_message,
        memory_summary=memory_summary,
        diagnostics=diagnostics
    )
    return {
        "title": "Quick clarification",
        "preview": body.split("\n")[0],
        "body": body,
        "quick_replies": [
            "Electrical resistance in circuits.",
            "Air resistance in motion.",
            "Give a short definition with one circuit example."
        ],
        "diagnostics": diagnostics or {},
        "memory_summary": memory_summary or ""
    }

In [49]:
def handle_user_message(user_message, history=None, prefer_generator=None):
    history = history or []

    user_message = rewrite_ambiguous_query_with_llm(user_message, history)

    pdf_context = get_relevant_context(user_message, 3)
    memory_context = get_relevant_memories(user_message, 2)

    if is_pdf_context_relevant(pdf_context, user_message):
        return {
            "allowed": True,
            "clarify": False,
            "answer": generate_answer(
                f"Context:\n{pdf_context}\n\nUser question: {user_message}",
                memory_summary=memory_context,
                history=history,
                pdf_context=pdf_context,
                prefer_generator=prefer_generator
            )[0]
        }

    ok, payload = vet_query(user_message)
    if not ok:
        return {"allowed": False, "reason": payload.get("reason")}

    if payload.get("clarify"):
        return {
            "allowed": True,
            "clarify": True,
            "clarification_payload": build_clarification_payload(
                user_message,
                payload.get("memory_summary", ""),
                payload.get("diagnostics", {})
            )
        }

    answer, raw, src = generate_answer(
        user_message,
        memory_summary=payload.get("memory_summary", ""),
        diagnostics=payload.get("diagnostics", {}),
        history=history,
        prefer_generator=prefer_generator
    )

    store_conversation_memory(user_message, answer, payload.get("memory_summary", ""))
    return {"allowed": True, "clarify": False, "answer": answer}

In [26]:
def accept_clarification_and_generate(clarification_text: str, original_user_message: str = None,
                                      diagnostics: dict = None, memory_summary: str = "", prefer_generator: str = None):
    """
    Treat clarification_text as the user's confirmed/expanded message and generate a final answer.
    - clarification_text: the text the user confirmed or selected from quick replies
    - original_user_message: optional, for logging/context
    - diagnostics, memory_summary: optional, passed through from vet_query
    Returns the same structure as handle_user_message but with clarify=False and final answer.
    """
    try:
        log_diagnostics_csv(user_message=original_user_message or clarification_text,
                            allowed=True, clarify=False, generator="clarify_confirm",
                            diagnostics=diagnostics or {}, answer_text=clarification_text, raw_reply={})
    except Exception:
        pass

    answer_text, raw_reply, source = generate_answer(
        user_message=clarification_text,
        memory_summary=memory_summary,
        diagnostics=diagnostics or {},
        clarify=False,
        prefer=prefer_generator
    )

    try:
        memory.store_interaction(user_message=clarification_text, assistant_reply=answer_text, diagnostics=diagnostics or {})
    except Exception:
        pass

    try:
        log_diagnostics_csv(user_message=clarification_text, allowed=True, clarify=False,
                            generator=source, diagnostics=diagnostics or {},
                            answer_text=answer_text, raw_reply=raw_reply)
    except Exception:
        pass

    return {
        "allowed": True,
        "clarify": False,
        "answer": answer_text,
        "raw_reply": raw_reply,
        "diagnostics": diagnostics or {},
        "memory_summary": memory_summary,
        "generator": source,
    }


In [27]:
CLARIFY_CONFIG = {
    "max_short_tokens": 6,
    "edu_lower": 0.30,
    "edu_upper": 0.38,
    "force_clarify_on_empty": True,
}

def decide_clarify(diagnostics: dict, len_tokens: int) -> bool:
    """
    Return True only when diagnostics indicate borderline educational intent
    or explicit ambiguity. Do not force clarify on empty diagnostics.
    """
    if not isinstance(diagnostics, dict) or not diagnostics:
        return False

    edu_score = float(diagnostics.get("edu_topk_mean", 0.0))
    if len_tokens <= CLARIFY_CONFIG["max_short_tokens"] and CLARIFY_CONFIG["edu_lower"] <= edu_score < CLARIFY_CONFIG["edu_upper"]:
        return True

    return False

In [28]:
from sentence_transformers import SentenceTransformer, util
import torch
import re
import numpy as np
from functools import lru_cache

# Model and precomputed embeddings (keep as you already have)
model = SentenceTransformer("all-MiniLM-L6-v2")

harmful_phrases = [
    "harmful", "dangerous", "illegal", "weapon", "explosive",
    "hack", "cheat", "bypass", "circumvent", "malicious"
]
educational_phrases = [
    "physics", "quantum", "mechanics", "thermodynamics", "electromagnetism",
    "resistance", "current", "voltage", "circuit", "ohm", "electricity",
    "relativity", "optics", "atomic", "nuclear", "particle", "force",
    "energy", "motion", "velocity", "acceleration", "gravity", "electric",
    "magnetic", "wave", "light", "sound", "heat", "temperature",
    "science", "math", "chemistry", "biology", "astronomy", "engineering",
    "technology", "computer", "programming", "learn", "study", "education",
    "explain", "how does", "what is", "why does"
]

# Precompute embeddings (ensure 2-D tensors and normalize)
harmful_embeds = model.encode(harmful_phrases, convert_to_tensor=True)  # shape (N_h, D)
educational_embeds = model.encode(educational_phrases, convert_to_tensor=True)  # shape (N_e, D)

# Normalize rows (works for both 1-D and 2-D safely)
def normalize_rows(t: torch.Tensor) -> torch.Tensor:
    if t.dim() == 1:
        t = t.unsqueeze(0)            # make (1, D)
        t = util.normalize_embeddings(t)
        return t.squeeze(0)          # back to (D,)
    else:
        return util.normalize_embeddings(t)  # (N, D)

harmful_embeds = normalize_rows(harmful_embeds)        # (N_h, D)
educational_embeds = normalize_rows(educational_embeds)  # (N_e, D)

# Simple negation detector used by check_educational_context
_negation_re = re.compile(
    r"\b(no|not|never|don't|dont|cannot|can't|cant|without|avoid|avoidance)\b",
    flags=re.IGNORECASE,
)

@lru_cache(maxsize=4096)
def _embed_text_cached(text: str) -> torch.Tensor:
    emb = model.encode(text, convert_to_tensor=True)  # (D,)
    if emb.dim() == 2 and emb.shape[0] == 1:
        emb = emb.squeeze(0)
    emb = normalize_rows(emb)
    return emb

@traceable_function
def check_educational_context(
    user_message: str,
    harmful_threshold: float = 0.45,
    educational_threshold: float = 0.35,
    top_k: int = 3,
    keyword_fallback_threshold: int = 1
):
    """
    Returns (allowed: bool, reason: str, diagnostics: dict)
    diagnostics contains scores for tuning and logging.
    """

    text = user_message.strip()
    if not text:
        return False, "Empty query"

    # Quick keyword fallback for very short queries
    tokens = re.findall(r"\w+", text.lower())
    if len(tokens) <= 3:
        # count educational keywords present (simple fallback)
        edu_hits = sum(1 for kw in educational_phrases if kw in text.lower())
        harm_hits = sum(1 for kw in harmful_phrases if kw in text.lower())
        if harm_hits >= keyword_fallback_threshold and not _negation_re.search(text):
            return False, "This short query contains harmful keywords"
        if edu_hits >= keyword_fallback_threshold:
            return True, "OK"

    # Embedding-based checks
    user_emb = _embed_text_cached(text)  # normalized
    harm_sims = util.cos_sim(user_emb, harmful_embeds).cpu().numpy().flatten()
    edu_sims = util.cos_sim(user_emb, educational_embeds).cpu().numpy().flatten()

    harm_sims = np.asarray(harm_sims)
    edu_sims = np.asarray(edu_sims)

    harm_topk_mean = float(np.sort(harm_sims)[-top_k:].mean())
    harm_max = float(harm_sims.max())
    edu_topk_mean = float(np.sort(edu_sims)[-top_k:].mean())
    edu_max = float(edu_sims.max())

    diagnostics = {
        "harm_topk_mean": harm_topk_mean,
        "harm_max": harm_max,
        "edu_topk_mean": edu_topk_mean,
        "edu_max": edu_max,
        "len_tokens": len(tokens)
    }

    # Harmful detection: prefer topk mean but also consider max
    if (harm_topk_mean >= harmful_threshold or harm_max >= harmful_threshold + 0.05) and not _negation_re.search(text):
        return False, "This query seems harmful or unsafe."

    # Educational detection
    if edu_topk_mean >= educational_threshold or edu_max >= educational_threshold + 0.05:
        return True, "OK"

    # Default: block with helpful message
    return False, "Please ask questions related to physics or education."

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
import yaml
from pathlib import Path

TEMPLATES_PATH = Path("prompts/templates.yml")

with open(TEMPLATES_PATH, "r", encoding="utf-8") as f:
    raw = yaml.safe_load(f)

# Build SystemMessagePromptTemplate
system_tpl = raw.get("system", {}).get("tutor", {}).get("content", "")
system_msg_template = SystemMessagePromptTemplate.from_template(system_tpl)

# Build HumanMessagePromptTemplate variants
user_default_tpl = raw.get("user", {}).get("default", {}).get("content", "")
user_clarify_tpl = raw.get("user", {}).get("clarification", {}).get("content", "")

human_default = HumanMessagePromptTemplate.from_template(user_default_tpl)
human_clarify = HumanMessagePromptTemplate.from_template(user_clarify_tpl)

# Compose a ChatPromptTemplate (example: system + human default)
chat_prompt_default = ChatPromptTemplate.from_messages([system_msg_template, human_default])

In [30]:
class _MemoryStub:
    def get_summary_for_user(self, *args, **kwargs):
        return ""

    def store_interaction(self, *args, **kwargs):
        return True

memory = _MemoryStub()


In [31]:
import logging
logging.basicConfig(level=logging.INFO)

def vet_query(user_message: str):
    ok, msg = apply_guardrails(user_message)
    if not ok:
        return False, {"reason": msg}

    result = check_educational_context(user_message)
    if isinstance(result, tuple) and len(result) == 3:
        allow, reason, diagnostics = result
    elif isinstance(result, tuple) and len(result) == 2:
        allow, reason = result
        diagnostics = {}
    else:
        raise ValueError("Unexpected return from check_educational_context")

    logging.info("context_check: allow=%s reason=%s diagnostics=%s", allow, reason, diagnostics)

    if not allow:
        return False, {"reason": reason, "diagnostics": diagnostics}

    pdf_context = ""
    try:
        pdf_context = get_relevant_context(user_message, n_results=3)
    except Exception:
        pdf_context = ""

    if pdf_context and is_pdf_context_relevant(pdf_context, user_message):
        return True, {"memory_summary": memory.get_summary_for_user() if hasattr(memory, "get_summary_for_user") else "", "diagnostics": diagnostics, "clarify": False}

    clarify = decide_clarify(diagnostics, diagnostics.get("len_tokens", 0))

    if (not diagnostics) and (not pdf_context):
        clarify = False

    return True, {"memory_summary": memory.get_summary_for_user() if hasattr(memory, "get_summary_for_user") else "", "diagnostics": diagnostics, "clarify": clarify}


In [32]:
def format_clarification_for_ui(user_message: str, memory_summary: str = "", diagnostics: dict = None, max_preview_chars: int = 240):
    body = prompt_mgr.render_clarify_user(
        original_query=user_message,
        memory_summary=memory_summary,
        diagnostics=diagnostics
    )
    preview = body.strip().split("\n\n")[0]
    if len(preview) > max_preview_chars:
        preview = preview[:max_preview_chars].rsplit(" ", 1)[0] + "…"

    quick_replies = [
        "Electrical resistance in circuits.",
        "Air resistance in motion.",
        "Give a short definition with one circuit example."
    ]

    return {
        "title": "Quick clarification",
        "preview": preview,
        "body": body,
        "quick_replies": quick_replies
    }

In [33]:
from flask import Flask, request, jsonify

def get_clarification_payload(user_message: str, memory_summary: str = "", diagnostics: dict = None):
    """
    Return a JSON-serializable dict with the clarification prompt and UI metadata.
    Use this from your frontend or call it inside your notebook.
    """
    body = prompt_mgr.render_clarify_user(
    original_query=user_message,
    memory_summary=memory_summary,
    diagnostics=diagnostics
)
    preview = body.strip().split("\n\n")[0]
    quick_replies = [
        "Can you give more context or an example?",
        "I mean the circuit in my homework, not a theoretical example.",
        "Show a short definition and one example."
    ]
    return {
        "title": "Quick clarification",
        "preview": preview,
        "body": body,
        "quick_replies": quick_replies,
        "diagnostics": diagnostics or {},
        "memory_summary": memory_summary or ""
    }

In [34]:
from typing import Dict

chat_prompt_clarify = ChatPromptTemplate.from_messages([system_msg_template, human_clarify])

class PromptManager:
    def __init__(self, default_prompt: ChatPromptTemplate, clarify_prompt: ChatPromptTemplate,
                 system_template: SystemMessagePromptTemplate = None, templates_yaml: dict = None):
        self.default = default_prompt
        self.clarify = clarify_prompt
        self.system_template = system_template
        self.templates = templates_yaml or {}

    def render_system(self, **kwargs) -> str:
        """
        Return a single system string. Use the explicit system template if available,
        otherwise fall back to the default system prompt object.
        """
        try:
            if self.system_template:
                pv = self.system_template.format_prompt(**kwargs)
                msgs = pv.to_messages()
                return msgs[0].content if msgs else ""
            pv = system_msg_template.format_prompt(**kwargs)
            msgs = pv.to_messages()
            return msgs[0].content if msgs else ""
        except Exception:
            return "You are a concise, patient physics tutor for high-school and early-college students."

    def render_user(self, user_message: str, clarify: bool = False) -> str:
        """
        Render the human message. Try LangChain rendering first; if the result
        still contains an unsubstituted placeholder like {user_message},
        perform a safe replacement on the final message content.
        """
        try:
            tpl = self.clarify if clarify else self.default
            rendered = tpl.format_prompt(user_message=user_message)
            msgs = rendered.to_messages()
            if not msgs:
                return user_message

            content = msgs[-1].content or ""
            if "{user_message}" in content or "{input}" in content or "{text}" in content or "{query}" in content:
                content = content.replace("{user_message}", user_message)
                content = content.replace("{input}", user_message)
                content = content.replace("{text}", user_message)
                content = content.replace("{query}", user_message)
            return content
        except Exception:
            return user_message



    def render_clarify_user(self, original_query: str, memory_summary: str = "", diagnostics: dict = None) -> str:
        """
        Produce a short, user-facing clarification prompt. Prefer a YAML template 'clarify_user'
        if present; otherwise return a concise default clarification text.
        This text is for the UI only and should not be sent to the model as a system instruction.
        """
        try:
            tmpl = self.templates.get("clarify_user")
            if tmpl:
                return tmpl.format(original_query=original_query, memory_summary=memory_summary or "")
        except Exception:
            pass

        return (
            f'Could you clarify what you mean by "{original_query}"?\n'
            "- electrical resistance in circuits (Ohm’s law)\n"
            "- air resistance (drag)\n"
            "- something else?\n\n"
            "Please choose one or add a brief detail."
        )

prompt_mgr = PromptManager(
    default_prompt=chat_prompt_default,
    clarify_prompt=chat_prompt_clarify,
    system_template=system_msg_template,
    templates_yaml=raw
)
print("✅ PromptManager ready")


✅ PromptManager ready


In [53]:
def build_messages(user_message, history, memory_summary, pdf_context, clarify=False):
    messages = [{"role": "system", "content": prompt_mgr.render_system()}]

    for turn in history[-6:]:
        if len(turn) >= 1:
            messages.append({"role": "user", "content": str(turn[0])})
        if len(turn) >= 2:
            messages.append({"role": "assistant", "content": str(turn[1])})

    if pdf_context or memory_summary:
        ctx = []
        if pdf_context:
            ctx.append("PDF context:\n" + pdf_context)
        if memory_summary:
            ctx.append("Memory summary:\n" + memory_summary)
        messages.append({"role": "system", "content": "\n\n---\n".join(ctx)})

    user_text = prompt_mgr.render_user(user_message, clarify=clarify)
    messages.append({"role": "user", "content": user_text})
    return messages

In [52]:
def call_generator(messages, prefer_generator=None):
    gen = (prefer_generator or GENERATOR_PREFERENCE).lower()
    if gen == "nemo":
        raw = nemo_rails.generate(messages=messages)
        return _extract_text_from_reply(raw), raw, "nemo"

    resp = openai.ChatCompletion.create(
        model=OPENAI_MODEL,
        messages=messages,
        temperature=0.0
    )
    return resp["choices"][0]["message"]["content"], resp, "openai"

In [51]:
GENERATOR_PREFERENCE = "nemo"
OPENAI_MODEL = "gpt-3.5-turbo"

def generate_answer(
    user_message,
    memory_summary="",
    diagnostics=None,
    clarify=False,
    history=None,
    pdf_context="",
    prefer_generator=None
):
    history = history or []
    diagnostics = diagnostics or {}

    messages = build_messages(
        user_message=user_message,
        history=history,
        memory_summary=memory_summary,
        pdf_context=pdf_context,
        clarify=clarify
    )

    answer, raw, source = call_generator(messages, prefer_generator)

    if answer.strip().lower().startswith(("hi", "hello", "hey")):
        messages.append({
            "role": "user",
            "content": user_message + "\n\nPlease answer directly."
        })
        answer, raw, _ = call_generator(messages, prefer_generator)

    return answer or "", raw, source


In [36]:
ans, raw, src = generate_answer("Explain electrical resistance briefly and give one circuit example.", history=[], memory_summary="", diagnostics={}, clarify=False)
print("Source:", src)
print("Answer snippet:", (ans or "")[:800])

=== MODEL MESSAGES ===
system : You are a concise, patient physics tutor for high-school and early-college students.
user : Explain electrical resistance briefly and give one circuit example.
Source: nemo
Answer snippet: Electrical resistance is a property that hinders the flow of electric current in a material. It is measured in ohms (Ω) and is influenced by factors like material type, length, and cross-sectional area.

One example of a circuit with resistance is a simple series circuit consisting of a battery, a resistor, and an LED. The resistor limits the current flowing through the LED, preventing it from burning out due to excessive current.


In [37]:
q = "What is resistance in current electricity?"
ans, raw, src = generate_answer(q, history=[], memory_summary="", diagnostics={}, clarify=False)
print("=== ANSWER ===")
print("Source:", src)
print("Answer snippet:", (ans or "")[:400])
print("=== RAW REPLY ===")
print(raw)

=== MODEL MESSAGES ===
system : You are a concise, patient physics tutor for high-school and early-college students.
user : What is resistance in current electricity?
=== ANSWER ===
Source: nemo
Answer snippet: Resistance is a property of a material that hinders the flow of electric current through it. It is measured in ohms (Ω) and is denoted by the symbol R. The higher the resistance of a material, the more difficult it is for electric current to pass through it. Resistance is influenced by factors such as the material of the conductor, its length, cross-sectional area, and temperature. Ohm's Law state
=== RAW REPLY ===
{'role': 'assistant', 'content': "Resistance is a property of a material that hinders the flow of electric current through it. It is measured in ohms (Ω) and is denoted by the symbol R. The higher the resistance of a material, the more difficult it is for electric current to pass through it. Resistance is influenced by factors such as the material of the conductor, it

In [38]:
system_chat = ChatPromptTemplate.from_messages([system_msg_template])

In [39]:
import openai

def answer_query_with_openai(user_message: str, memory_summary: str = "", diagnostics: dict = None, clarify: bool = False):
    """
    Render system + user via prompt_mgr and call OpenAI ChatCompletion.
    Returns (assistant_text: str, raw_reply: dict)
    """
    system_text = prompt_mgr.render_system()
    user_text = prompt_mgr.render_user(user_message, clarify=clarify, memory_summary=memory_summary, diagnostics=diagnostics)

    messages = [
        {"role": "system", "content": system_text},
        {"role": "user", "content": user_text},
    ]

    try:
        resp = openai.ChatCompletion.create(
            model="gpt-3.5-turbo" ,
            messages=messages,
            temperature=0.0,
            max_tokens=2500,
        )
        choice = resp["choices"][0]["message"]
        assistant_text = choice.get("content", "")
        return assistant_text, resp
    except Exception as e:
        return f"Error generating response via OpenAI: {e}", {"error": str(e)}


In [41]:
@traceable_function
def clear_memory():
    """Clear all conversation memory"""
    try:
        memory_collection.delete(where={"type": {"$ne": ""}})

        # Log memory clearance
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=None,
                    key="memory_cleared",
                    score=1.0,
                    comment="All conversation memory cleared"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return "✅ Conversation memory cleared successfully!"
    except Exception as e:
        return f"❌ Error clearing memory: {str(e)}"

In [42]:
@traceable_function
def export_memory():
    """Export conversation memory as JSON"""
    try:
        results = memory_collection.get()
        memories = []

        for i, doc in enumerate(results['documents']):
            memories.append({
                "content": doc,
                "metadata": results['metadatas'][i],
                "id": results['ids'][i]
            })

        filename = f"conversation_memory_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w') as f:
            json.dump(memories, f, indent=2)

        # Log export
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=None,
                    key="memory_exported",
                    score=1.0,
                    comment=f"Exported {len(memories)} memories to {filename}"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return f"✅ Memory exported to {filename}"
    except Exception as e:
        return f"❌ Error exporting memory: {str(e)}"

In [43]:
def get_guardrails_status():
    """Get status of safety guardrails"""
    if NEMO_GUARDRAILS_AVAILABLE and nemo_rails is not None:
        return "✅ NVIDIA NeMo Guardrails: ACTIVE"
    else:
        return "⚠️ NVIDIA NeMo Guardrails: NOT AVAILABLE"

def get_token_usage():
    """Get current token usage statistics"""
    summary = token_tracker.get_summary()
    return f"""
📊 Token Usage Summary:
• Total Tokens: {summary['total_tokens']:,}
• Prompt Tokens: {summary['prompt_tokens']:,}
• Completion Tokens: {summary['completion_tokens']:,}
• Estimated Cost: ${summary['estimated_cost']}
"""

In [44]:
def reset_token_tracking():
    """Reset token tracking counters"""
    global token_tracker
    token_tracker = TokenUsageTracker()
    return "✅ Token tracking reset successfully!"

In [45]:
def get_langsmith_status():
    """Get LangSmith tracing status"""
    if LANGCHAIN_AVAILABLE:
        return "✅ LangSmith Tracing: ACTIVE"
    else:
        return "⚠️ LangSmith Tracing: NOT AVAILABLE"

In [46]:
def load_pdf_and_chat(pdf_file, message, history):
    result_msg = store_pdf_in_db(pdf_file.name)
    response = handle_user_message(message, history)

    return result_msg, response

In [47]:
_pending_clarify = {}

def _history_key(history):
    try:
        return str(history[-6:])
    except Exception:
        return repr(history)

def _is_substantive_answer(text: str, min_len: int = 30) -> bool:
    if not text:
        return False
    t = text.strip().lower()
    if len(t) < min_len:
        return False
    for prefix in ("hello", "hi", "hey", "i'm here to help", "feel free to ask"):
        if t.startswith(prefix):
            return False
    return True

def _last_entry_equals(history, user, assistant):
    if not history:
        return False
    last = history[-1]
    return len(last) >= 2 and last[0] == user and last[1] == assistant

def gradio_adapter(message, history):
    """
    Adapter for Gradio ChatInterface:
    - If a clarification is pending for this history, treat `message` as the confirmation text.
    - Otherwise call handle_user_message(message, history) and return a string.
    """
    print("gradio_adapter called:", message, "history_len:", len(history))
    global _pending_clarify
    key = _history_key(history)

    if key in _pending_clarify:
        pending = _pending_clarify.pop(key)
        final = accept_clarification_and_generate(
            clarification_text=message,
            original_user_message=pending.get("original_query"),
            diagnostics=pending.get("diagnostics", {}),
            memory_summary=pending.get("memory_summary", ""),
            history=history
        )
        answer = final.get("answer") or ""
        if _is_substantive_answer(answer):
            history.append([message, answer])
        return answer or "No answer generated."

    result = handle_user_message(message, history=history)

    if not isinstance(result, dict):
        reply = str(result)
        if _is_substantive_answer(reply):
            history.append([message, reply])
        return reply

    if result.get("clarify"):
        _pending_clarify[key] = {
            "original_query": message,
            "diagnostics": result.get("diagnostics", {}),
            "memory_summary": result.get("memory_summary", "")
        }
        payload = result.get("clarification_payload", {})
        return payload.get("body") or payload.get("preview") or "Could you clarify?"

    answer = result.get("answer") or _extract_text_from_reply(result.get("raw_reply"))
    raw = result.get("raw_reply")

    return answer or "I didn’t catch that clearly. Could you rephrase or be more specific?"

In [55]:
def launch_app():
    with gr.Blocks() as demo:
        gr.Markdown("# ⚛️ Physics Teacher Chat with PDF RAG + Web Search + Memory + Safety Guardrails + LangSmith Tracing")

        with gr.Row():
            with gr.Column():
                pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
                load_btn = gr.Button("Load PDF into Knowledge Base")
                load_status = gr.Textbox(label="Load Status", interactive=False)

                gr.Markdown("### Safety & Memory Management")
                with gr.Row():
                    clear_mem_btn = gr.Button("🧹 Clear Memory")
                    export_mem_btn = gr.Button("💾 Export Memory")
                    safety_status_btn = gr.Button("🛡️ Safety Status")
                mem_status = gr.Textbox(label="Memory Status", interactive=False)

                gr.Markdown("### Token Usage Tracking")
                with gr.Row():
                    token_stats_btn = gr.Button("📊 Token Usage")
                    reset_tokens_btn = gr.Button("🔄 Reset Tracking")
                    tracing_status_btn = gr.Button("🔍 Tracing Status")
                token_status = gr.Textbox(label="Token Status", interactive=False)

                gr.Markdown("""
                ### How it works:
                1. **Upload a PDF** - Answers prioritize PDF content when relevant
                2. **Ask questions** - System uses PDF, web search, and past conversations
                3. **Long-term memory** - Remembers past conversations for context
                4. **Safety Guardrails** - NVIDIA NeMo Guardrails ensure safe, educational responses
                5. **Multiple sources** - Combines PDF 📚, Web 🔍, and Memory 💭
                6. **LangSmith Tracing** - Comprehensive tracing and token usage tracking
                7. **Token Monitoring** - Real-time token usage and cost tracking
                """)

            with gr.Column():
                chat_interface = gr.ChatInterface(
                    fn=gradio_adapter,
                    title="Chat with Physics Bot (with Memory, Safety & Tracing)",
                    description="Ask questions about physics. I'll remember our past conversations, use multiple knowledge sources, ensure safe responses, and track token usage!"
                )

        # Load PDF when button clicked
        load_btn.click(
            fn=lambda pdf: store_pdf_in_db(pdf.name) if pdf else "Please upload a PDF first",
            inputs=[pdf_input],
            outputs=[load_status]
        )

        # Memory management buttons
        clear_mem_btn.click(
            fn=clear_memory,
            outputs=[mem_status]
        )

        export_mem_btn.click(
            fn=export_memory,
            outputs=[mem_status]
        )

        safety_status_btn.click(
            fn=get_guardrails_status,
            outputs=[mem_status]
        )

        # Token tracking buttons
        token_stats_btn.click(
            fn=get_token_usage,
            outputs=[token_status]
        )

        reset_tokens_btn.click(
            fn=reset_token_tracking,
            outputs=[token_status]
        )

        tracing_status_btn.click(
            fn=get_langsmith_status,
            outputs=[token_status]
        )

    demo.launch()

if __name__ == "__main__":
    # Check for required API keys
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OPENAI_API_KEY not found in environment variables")
        exit(1)

    if not os.getenv("TAVILY_API_KEY") and TAVILY_AVAILABLE:
        print("❌ TAVILY_API_KEY not found in environment variables")
        print("Please get your free API key from: https://app.tavily.com/")
        print("Continuing without web search functionality...")

    # Check for LangSmith API key
    if not os.getenv("LANGCHAIN_API_KEY"):
        print("⚠️  LANGCHAIN_API_KEY not found. LangSmith tracing will be disabled.")
        print("💡 Get your free API key from: https://smith.langchain.com/")

    # Auto-load PDF if exists
    pdf_files = [f for f in os.listdir('.') if f.endswith('.pdf')]
    if pdf_files:
        print(f"Found PDF: {pdf_files[0]}")
        store_pdf_in_db(pdf_files[0])

    print("🧠 Long-term memory enabled!")
    print("📊 Token usage tracking enabled!")
    print(get_guardrails_status())
    print(get_langsmith_status())

    if LANGCHAIN_AVAILABLE:
        print("🔍 LangSmith tracing enabled! Visit https://smith.langchain.com/ to view traces.")

    launch_app()

🧠 Long-term memory enabled!
📊 Token usage tracking enabled!
✅ NVIDIA NeMo Guardrails: ACTIVE
✅ LangSmith Tracing: ACTIVE
🔍 LangSmith tracing enabled! Visit https://smith.langchain.com/ to view traces.


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd181c1a915b725df8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [56]:
gradio_adapter("What is resistance?", [])
gradio_adapter("What is resistance in current electricity?", [])
gradio_adapter("Compare resistance and resistivity.", [])

gradio_adapter called: What is resistance? history_len: 0
gradio_adapter called: What is resistance in current electricity? history_len: 0
gradio_adapter called: Compare resistance and resistivity. history_len: 0


"Resistance is a property of an object or material that hinders the flow of electric current through it. It is measured in ohms and depends on the material, length, and cross-sectional area of the object.\n\nResistivity, on the other hand, is an intrinsic property of a material that quantifies how strongly it resists the flow of electric current. It is denoted by the symbol ρ and is measured in ohm-meters. Resistivity is a characteristic of the material itself and is independent of the object's dimensions.\n\nIn summary, resistance depends on the object's dimensions, while resistivity is an inherent property of the material itself."

In [57]:
def _debug_handle_user_message(user_message):
    print("=== DEBUG handle_user_message start ===")
    print("user_message:", repr(user_message))

    pdf_context = get_relevant_context(user_message, n_results=3)
    print("pdf_context (len):", len(pdf_context))
    print("is_pdf_context_relevant:", is_pdf_context_relevant(pdf_context, user_message))

    ok, payload = vet_query(user_message)
    print("vet_query -> ok:", ok, "payload:", payload)

    if ok and not payload.get("clarify", False):
        print("Proceeding to generate_answer() now...")
        memory_summary = payload.get("memory_summary", "")
        diagnostics = payload.get("diagnostics", {})
        ans_text, raw, source = generate_answer(user_message, memory_summary=memory_summary, diagnostics=diagnostics, clarify=False)
        print("generate_answer returned (snippet):", (ans_text or "")[:400])
        print("raw reply type:", type(raw))
        print("generator source:", source)
    else:
        print("Clarify required or vet_query blocked; payload clarify:", payload.get("clarify"))

    print("=== DEBUG handle_user_message end ===")
    return ok, payload

_debug_handle_user_message("resistance?")

=== DEBUG handle_user_message start ===
user_message: 'resistance?'
pdf_context (len): 0
is_pdf_context_relevant: False
vet_query -> ok: True payload: {'memory_summary': '', 'diagnostics': {}, 'clarify': False}
Proceeding to generate_answer() now...
generate_answer returned (snippet): Resistance is a measure of how much a material or device opposes the flow of electric current. It is denoted by the symbol \( R \) and is measured in ohms (\( \Omega \)). The higher the resistance, the more difficult it is for current to flow through the material or device. Resistance can be calculated using Ohm's Law, which states that \( R = \frac{V}{I} \), where \( R \) is resistance, \( V \) i
raw reply type: <class 'dict'>
generator source: nemo
=== DEBUG handle_user_message end ===


(True, {'memory_summary': '', 'diagnostics': {}, 'clarify': False})